The overall goal is to create a Python script that makes a custom model for emotional recognition and deploys it live to a camera.
- The structure of the data is a subset of folders containing images. The folders are labeled by their respective motions: Ahegao, Angry, Happy, Neutral, Sad, Surprise. 
- Convert the RGB images into a numpy array.
- When creating the CNN architecture, add Ridge Regression regularization to each layer. Add Dropout and BatchNormalization layers.
- When training the CNN add Kfold cross validation.
- Make sure the script is using an Nvidia GPU
- Save the best performing checkpoint and label the model as “emotional_recognition_val_accuracy_{val_accuracy:.2f}”
- Then write a script in Python, referring to the prior script, that pre processes the live feed of the 

**More Details Here**
https://chat.openai.com/share/6aca2c47-efd4-41a6-83cd-9eabe7b2d4fd

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
#Best hyperparameters: {'batch_size': 16, 'dropout_val': 0.3, 'epochs': 10, 'l2_val': 0.001, 'num_folds': 10}

In [3]:
import os
import numpy as np
import cv2
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid

#Parameters
num_classes = 7
img_width, img_height = 48, 48 #Resize all images to this size. Ensure minimum resolution is not less than this.
min_res = (img_width, img_height) # Update to set minimum resolution of photos to train on 
max_res = (512, 512) # Update to set maximum resolution of photos to train on 

def load_data(data_dir, min_resolution=None, max_resolution=None):
    images = []
    labels = []
    emotions = ["angry", "disgust", "fear", "happy", "neutral", "sad","surprise"]
    
    for idx, emotion in enumerate(emotions):
        emotion_folder = os.path.join(data_dir, emotion)
        for img_file in os.listdir(emotion_folder):
            img_path = os.path.join(emotion_folder, img_file)
            image = cv2.imread(img_path)
            height, width, _ = image.shape
            #img = load_img(img_path, color_mode='rgb', target_size=(128, 128))  # Load image in RGB
            #img_gray = cv2.cvtColor(img_to_array(img), cv2.COLOR_RGB2GRAY)  # Convert to grayscale
            #img_rgb_gray = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2RGB)  # Convert back to RGB
            
            # Check image resolution
            if (min_resolution is None or (height >= min_resolution[0] and width >= min_resolution[1])) and \
               (max_resolution is None or (height <= max_resolution[0] and width <= max_resolution[1])):
                image = cv2.resize(image, (img_width, img_height))
                images.append(image)
                labels.append(idx)
    return np.array(images), np.array(labels)




# Load and preprocess data
data_directory = r'C:\Important Files\Python Projects\Facial Emotion Recognition\FER-2013\dataset'  # Replace this with your dataset directory
X, y = load_data(data_directory, min_resolution=min_res, max_resolution=max_res)
X = X / 255.0  # Normalize pixel values

# Convert labels to categorical
y = to_categorical(y, num_classes=num_classes)  # 7 classes for the 7 emotions

# Define the hyperparameters grid for grid search
#Best hyperparameters: {'batch_size': 16, 'dropout_val': 0.3, 'epochs': 5, 'l2_val': 0.01, 'num_folds': 10}
param_grid = {
    'batch_size': [16, 32, 64],
    'l2_val': [0.1, 0.01, 0.001],
    'dropout_val': [0.7, 0.5, 0.3],
    'epochs': [5, 10, 15],
    'num_folds': [2, 5, 10]
}

# Define the CNN model with Ridge regularization
def create_model(l2_val, dropout_val):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3), kernel_regularizer=l2(l2_val)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(l2_val)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(l2_val)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_val))
    model.add(Dense(7, activation='softmax'))  # Output layer with 7 classes
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

best_val_acc = 0.0
best_epoch = 0
best_params = {}

# Perform grid search
for params in ParameterGrid(param_grid):
    print(f"Training with hyperparameters: {params}")
    
    batch_size = params['batch_size']
    l2_val = params['l2_val']
    dropout_val = params['dropout_val']
    epochs = params['epochs']
    num_folds = params['num_folds']
    
    # Perform K-fold cross-validation
    kfold = KFold(n_splits=num_folds, shuffle=True)
    fold_no = 1
    
    for train_idx, val_idx in kfold.split(X, y):
        model = create_model(l2_val, dropout_val)
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        checkpoint_path ='emotional_CNN_val_accuracy_{val_accuracy:.2f}.h5'
        checkpoint = ModelCheckpoint(checkpoint_path,
                                     monitor='val_accuracy', verbose=1,
                                     save_best_only=True, mode='max')
        
        history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                            validation_data=(X_val, y_val), callbacks=[checkpoint], verbose=1)
        
        val_acc = max(history.history['val_accuracy'])
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_epoch = np.argmax(history.history['val_accuracy']) + 1
            best_params = params
    
        fold_no += 1

print(f"Best validation accuracy: {best_val_acc} at epoch {best_epoch}")
print("Best hyperparameters:", best_params)



# Now you can deploy the best model to recognize emotions from your camera
# Load the best model using model = load_model('best_model.h5') and perform predictions on live camera feed
# Remember to preprocess the camera feed similar to how the training data was preprocessed

Training with hyperparameters: {'batch_size': 16, 'dropout_val': 0.7, 'epochs': 5, 'l2_val': 0.1, 'num_folds': 2}
Epoch 1/5
1122/1122 [==============================] - 14s 8ms/step - loss: 2.6625 - accuracy: 0.2748 - val_loss: 1.8729 - val_accuracy: 0.2553

Epoch 00001: val_accuracy improved from -inf to 0.25529, saving model to emotional_CNN_val_accuracy_0.26.h5
Epoch 2/5
1122/1122 [==============================] - 9s 8ms/step - loss: 1.8101 - accuracy: 0.2972 - val_loss: 1.8366 - val_accuracy: 0.2859

Epoch 00002: val_accuracy improved from 0.25529 to 0.28589, saving model to emotional_CNN_val_accuracy_0.29.h5
Epoch 3/5
1122/1122 [==============================] - 9s 8ms/step - loss: 1.8046 - accuracy: 0.3001 - val_loss: 1.8294 - val_accuracy: 0.3001

Epoch 00003: val_accuracy improved from 0.28589 to 0.30010, saving model to emotional_CNN_val_accuracy_0.30.h5
Epoch 4/5
1122/1122 [==============================] - 10s 9ms/step - loss: 1.7875 - accuracy: 0.3121 - val_loss: 1.9648 - 

Epoch 1/5
2019/2019 [==============================] - 9s 4ms/step - loss: 2.3387 - accuracy: 0.2957 - val_loss: 2.3344 - val_accuracy: 0.1226

Epoch 00001: val_accuracy improved from -inf to 0.12260, saving model to emotional_CNN_val_accuracy_0.12.h5
Epoch 2/5
2019/2019 [==============================] - 10s 5ms/step - loss: 1.8042 - accuracy: 0.3284 - val_loss: 2.0315 - val_accuracy: 0.3082

Epoch 00002: val_accuracy improved from 0.12260 to 0.30816, saving model to emotional_CNN_val_accuracy_0.31.h5
Epoch 3/5
2019/2019 [==============================] - 11s 5ms/step - loss: 1.7825 - accuracy: 0.3352 - val_loss: 2.2427 - val_accuracy: 0.1669

Epoch 00003: val_accuracy did not improve from 0.30816
Epoch 4/5
2019/2019 [==============================] - 11s 5ms/step - loss: 1.7773 - accuracy: 0.3390 - val_loss: 1.8338 - val_accuracy: 0.3051

Epoch 00004: val_accuracy did not improve from 0.30816
Epoch 5/5
2019/2019 [==============================] - 11s 5ms/step - loss: 1.7771 - accurac

2019/2019 [==============================] - 9s 4ms/step - loss: 1.7697 - accuracy: 0.3055 - val_loss: 1.7556 - val_accuracy: 0.3216

Epoch 00003: val_accuracy did not improve from 0.33138
Epoch 4/5
2019/2019 [==============================] - 9s 4ms/step - loss: 1.7659 - accuracy: 0.3070 - val_loss: 1.7507 - val_accuracy: 0.2940

Epoch 00004: val_accuracy did not improve from 0.33138
Epoch 5/5
2019/2019 [==============================] - 9s 4ms/step - loss: 1.7646 - accuracy: 0.3093 - val_loss: 1.7585 - val_accuracy: 0.3180

Epoch 00005: val_accuracy did not improve from 0.33138
Epoch 1/5
2019/2019 [==============================] - 9s 4ms/step - loss: 2.2955 - accuracy: 0.2502 - val_loss: 1.8280 - val_accuracy: 0.2506

Epoch 00001: val_accuracy improved from -inf to 0.25056, saving model to emotional_CNN_val_accuracy_0.25.h5
Epoch 2/5
2019/2019 [==============================] - 9s 4ms/step - loss: 1.7919 - accuracy: 0.2844 - val_loss: 1.7745 - val_accuracy: 0.3055

Epoch 00002: val_


Epoch 00003: val_accuracy did not improve from 0.35391
Epoch 4/5
1795/1795 [==============================] - 8s 5ms/step - loss: 1.6823 - accuracy: 0.3770 - val_loss: 1.5643 - val_accuracy: 0.4415

Epoch 00004: val_accuracy improved from 0.35391 to 0.44155, saving model to emotional_CNN_val_accuracy_0.44.h5
Epoch 5/5
1795/1795 [==============================] - 8s 5ms/step - loss: 1.6758 - accuracy: 0.3808 - val_loss: 2.0026 - val_accuracy: 0.2327

Epoch 00005: val_accuracy did not improve from 0.44155
Epoch 1/5
1795/1795 [==============================] - 9s 5ms/step - loss: 2.0786 - accuracy: 0.3007 - val_loss: 2.0374 - val_accuracy: 0.2284

Epoch 00001: val_accuracy improved from -inf to 0.22837, saving model to emotional_CNN_val_accuracy_0.23.h5
Epoch 2/5
1795/1795 [==============================] - 8s 5ms/step - loss: 1.7582 - accuracy: 0.3371 - val_loss: 1.8026 - val_accuracy: 0.3031

Epoch 00002: val_accuracy improved from 0.22837 to 0.30305, saving model to emotional_CNN_val_

2019/2019 [==============================] - 9s 4ms/step - loss: 1.7032 - accuracy: 0.3777 - val_loss: 1.6478 - val_accuracy: 0.3892

Epoch 00003: val_accuracy improved from 0.38423 to 0.38924, saving model to emotional_CNN_val_accuracy_0.39.h5
Epoch 4/5
2019/2019 [==============================] - 9s 4ms/step - loss: 1.6933 - accuracy: 0.3866 - val_loss: 1.5856 - val_accuracy: 0.4589

Epoch 00004: val_accuracy improved from 0.38924 to 0.45890, saving model to emotional_CNN_val_accuracy_0.46.h5
Epoch 5/5
2019/2019 [==============================] - 9s 4ms/step - loss: 1.6823 - accuracy: 0.3912 - val_loss: 1.6981 - val_accuracy: 0.3945

Epoch 00005: val_accuracy did not improve from 0.45890
Epoch 1/5
2019/2019 [==============================] - 9s 4ms/step - loss: 2.0651 - accuracy: 0.3272 - val_loss: 1.7789 - val_accuracy: 0.3633

Epoch 00001: val_accuracy improved from -inf to 0.36333, saving model to emotional_CNN_val_accuracy_0.36.h5
Epoch 2/5
2019/2019 [============================

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [5]:
# Live feed preprocessing script should be implemented to process the live camera frames using OpenCV
# and feed them into the trained model for inference
# This involves capturing frames from the camera, preprocessing them similarly to the training data,
# and performing inference using the trained model.
# You can use OpenCV's VideoCapture to access the live camera feed.

# Load the trained model
model = load_model('emotional_CNN_val_accuracy_0.52.h5')  # Load your best model here

# Define emotions for mapping the output
emotions = ["angry", "disgust", "fear", "happy", "neutral", "sad","surprise"]

# Function to preprocess live feed frames
def preprocess_live_frame(frame):
    # Resize the frame to match the input size of the trained model
    resized_frame = cv2.resize(frame, (img_width, img_height))
    frame_normalized = resized_frame / 255.0  # Normalize pixel values
    return np.expand_dims(frame_normalized, axis=0)

# Access the live camera feed
cap = cv2.VideoCapture(0)  # Use 0 for the default camera, you may need to change this number

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Preprocess the live frame
    processed_frame = preprocess_live_frame(frame)
    
    # Perform inference using the trained model
    predictions = model.predict(processed_frame)
    confidence = np.max(predictions)
    emotion_label = emotions[np.argmax(predictions)]
    
    # Display recognized emotion and confidence level on the camera feed
    text = f"Emotion: {emotion_label}, Confidence: {confidence:.2f}"
    cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
    # Show the camera feed with emotion and confidence
    cv2.imshow('Emotion Recognition', frame)
    
    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()
